# Best practices for the real world

## Getting the most out of your models

### Hyperparameter optimization

#### Using KerasTuner

In [1]:
!pip install keras-tuner -q

     |████████████████████████████████| 135 kB 7.2 MB/s 
     |████████████████████████████████| 1.6 MB 44.6 MB/s 


**A KerasTuner model-building function**

In [2]:
from tensorflow import keras
from tensorflow.keras import layers

def build_model(hp):
    units = hp.Int(name="units", min_value=16, max_value=64, step=16)
    model = keras.Sequential([
        layers.Dense(units, activation="relu"),
        layers.Dense(10, activation="softmax")
    ])
    optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"])
    return model

**A KerasTuner `HyperModel`**

In [3]:
import kerastuner as kt

class SimpleMLP(kt.HyperModel):
    def __init__(self, num_classes):
        self.num_classes = num_classes

    def build(self, hp):
        units = hp.Int(name="units", min_value=16, max_value=64, step=16)
        model = keras.Sequential([
            layers.Dense(units, activation="relu"),
            layers.Dense(self.num_classes, activation="softmax")
        ])
        optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
        model.compile(
            optimizer=optimizer,
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"])
        return model

hypermodel = SimpleMLP(num_classes=10)

<ipython-input-3-643dcc05e516>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [4]:
tuner = kt.BayesianOptimization(
    build_model,
    objective="val_accuracy",
    max_trials=100,
    executions_per_trial=2,
    directory="mnist_kt_test",
    overwrite=True,
)

In [5]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': None}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}


In [6]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape((-1, 28 * 28)).astype("float32") / 255
x_test = x_test.reshape((-1, 28 * 28)).astype("float32") / 255
x_train_full = x_train[:]
y_train_full = y_train[:]
num_val_samples = 10000
x_train, x_val = x_train[:-num_val_samples], x_train[-num_val_samples:]
y_train, y_val = y_train[:-num_val_samples], y_train[-num_val_samples:]
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5),
]
tuner.search(
    x_train, y_train,
    batch_size=128,
    epochs=100,
    validation_data=(x_val, y_val),
    callbacks=callbacks,
    verbose=2,
)

Trial 100 Complete [00h 00m 48s]
val_accuracy: 0.9734500050544739

Best val_accuracy So Far: 0.9763000011444092
Total elapsed time: 01h 37m 31s


**Querying the best hyperparameter configurations**

In [7]:
top_n = 4
best_hps = tuner.get_best_hyperparameters(top_n)

In [12]:
def get_best_epoch(hp):
    model = build_model(hp)
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor="val_loss", mode="min", patience=10)
    ]
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=100,
        batch_size=128,
        callbacks=callbacks)
    val_loss_per_epoch = history.history["val_loss"]
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
    print(f"Best epoch: {best_epoch}")
    return best_epoch

In [16]:
model = build_model(hp)
def get_best_trained_model(hp):
    best_epoch = get_best_epoch(hp)
    model.fit(
        x_train_full, y_train_full,
        batch_size=128, epochs=int(best_epoch * 1.2))
    return model

best_models = []
for hp in best_hps:
    model = get_best_trained_model(hp)
    model.evaluate(x_test, y_test)
    best_models.append(model)

Epoch 1/100
391/391 [==============================] - 2s 4ms/step - loss: 0.4274 - accuracy: 0.8843 - val_loss: 0.2377 - val_accuracy: 0.9335
Epoch 2/100
391/391 [==============================] - 1s 3ms/step - loss: 0.2225 - accuracy: 0.9364 - val_loss: 0.1881 - val_accuracy: 0.9479
Epoch 3/100
391/391 [==============================] - 1s 3ms/step - loss: 0.1719 - accuracy: 0.9510 - val_loss: 0.1534 - val_accuracy: 0.9579
Epoch 4/100
391/391 [==============================] - 1s 3ms/step - loss: 0.1394 - accuracy: 0.9600 - val_loss: 0.1379 - val_accuracy: 0.9610
Epoch 5/100
391/391 [==============================] - 2s 4ms/step - loss: 0.1175 - accuracy: 0.9664 - val_loss: 0.1200 - val_accuracy: 0.9657
Epoch 6/100
391/391 [==============================] - 1s 3ms/step - loss: 0.1002 - accuracy: 0.9714 - val_loss: 0.1186 - val_accuracy: 0.9657
Epoch 7/100
391/391 [==============================] - 1s 4ms/step - loss: 0.0880 - accuracy: 0.9746 - val_loss: 0.1130 - val_accuracy: 0.9669

In [17]:
best_models = tuner.get_best_models(top_n)

#### The art of crafting the right search space

#### The future of hyperparameter tuning: automated machine learning

### Model ensembling

## Scaling-up model training

### Speeding up training on GPU with mixed precision

#### Understanding floating-point precision

In [18]:
import tensorflow as tf
import numpy as np
np_array = np.zeros((2, 2))
tf_tensor = tf.convert_to_tensor(np_array)
tf_tensor.dtype

tf.float64

In [19]:
np_array = np.zeros((2, 2))
tf_tensor = tf.convert_to_tensor(np_array, dtype="float32")
tf_tensor.dtype

tf.float32

#### Mixed-precision training in practice

In [20]:
from tensorflow import keras
keras.mixed_precision.set_global_policy("mixed_float16")

### Multi-GPU training

#### Getting your hands on two or more GPUs

#### Single-host, multi-device synchronous training

### TPU training

#### Using a TPU via Google Colab

#### Leveraging step fusing to improve TPU utilization

## Summary